### Mistral-7B-Instruct-v0.1 Finetuning 실습
- 한국어 기사-요약 데이터셋으로 미세 조정하여 한국어 기사를 요약하는 모델을 생성하고 ROUGE 지표로 평가하기
- Instruct Following 형식으로 학습 데이터 정제 및 SFT Trainer(지도학습 파인튜닝)로 학습시키기
- 데이터셋 출처: https://huggingface.co/datasets/daekeun-ml/naver-news-summarization-ko

### 1. 데이터 준비

In [ ]:
# 데이터셋 로드 및 샘플링
!pip install datasets -qqq
from datasets import load_dataset
train_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split='train').shuffle().select(range(2000))
eval_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split='validation').shuffle().select(range(200))
test_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split='test').shuffle().select(range(200))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/66.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/7.45M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/8.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

In [ ]:
# 불필요한 컬럼 제거
train_dataset = train_dataset.remove_columns(['date', 'category', 'press', 'title', 'link'])
eval_dataset = eval_dataset.remove_columns(['date', 'category', 'press', 'title', 'link'])
test_dataset = test_dataset.remove_columns(['date', 'category', 'press', 'title', 'link'])

In [ ]:
print(train_dataset, eval_dataset, test_dataset)

Dataset({
    features: ['document', 'summary'],
    num_rows: 2000
}) Dataset({
    features: ['document', 'summary'],
    num_rows: 200
}) Dataset({
    features: ['document', 'summary'],
    num_rows: 200
})


### 양자화 / 로라 설정 후 모델(양자화 장착) 및 토크나이저 불러오기

In [ ]:
# 허깅페이스 로그인
from huggingface_hub import login

# 허깅페이스 허브 로그인
token = "****"  # 허깅페이스 액세스 토큰 입력
login(token=token)

In [ ]:
!pip install bitsandbytes -qqq
import torch
import bitsandbytes
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# LoRA 설정
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],
    init_lora_weights="gaussian"
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.5 MB/s eta 0:00:00


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# 양자화 모델 준비
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", torch_dtype=torch.float16)  # 양자화 장착해서 모델 불러오기

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# 토크나이저 설정 및 준비
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token  # eos 토큰을 패딩 토큰으로 설정
tokenizer.padding_side = 'right'  # 학습 시 패딩 방향 '오른쪽'

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

### 데이터셋 대화형 포맷으로 변환 및 토큰화 (전처리)

In [ ]:
# instruction 템플릿 정의
def create_prompt_template(document):
    return f"""아래 뉴스 기사를 요약해주세요:

기사 내용:
{document}

요약: """

def create_completion_template(summary):
    return f"{summary}\n"

# 데이터 전처리 함수
def preprocess_function(examples):
    prompts = [create_prompt_template(doc) for doc in examples["document"]]
    completions = [create_completion_template(summary) for summary in examples["summary"]]  # instruction과 completion 생성

    # 전체 텍스트 생성 (instruction + completion)
    texts = [f"[INST]{prompt}[/INST]{completion}" for prompt, completion in zip(prompts, completions)]

    # 토큰화
    tokenized = tokenizer(texts, padding="max_length", truncation=True, max_length=512)

    # labels 설정 (input_ids와 동일)
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

# 데이터셋 전처리 적용
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(preprocess_function, batched=True, remove_columns=eval_dataset.column_names)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset, eval_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
}) Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 200
})


### 학습 어규먼트 설정 및 모델 학습하기

In [ ]:
# wandb 설정
import wandb
wandb.login(key='77a08abc9cfd76e4b78603826bd7a863487240ac')
run = wandb.init(project='Korean News Summarization', job_type='training', anonymous='allow')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dgriii0606 (dg-test) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
# 콜레이터 설정
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)  # Causal LM 모델용 콜레이터 # MLM이 아니라 CLM 방식이므로 False 설정

In [ ]:
!pip install trl -qqq
from trl import SFTTrainer
from transformers import TrainingArguments

# 트레이너 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,

    # 배치 크기 및 그래디언트 누적
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,

    # 학습률 및 스케줄링
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,

    # 옵티마이저 설정
    optim="adamw_torch",
    weight_decay=0.01,

    # 평가 및 저장 전략
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,

    # 학습 최적화
    fp16=True,
    gradient_checkpointing=True,

    # 로깅 설정
    logging_dir='./logs',
    logging_steps=10,
    report_to=["wandb"],

    # 기타 최적화
    dataloader_num_workers=2,     # 워커 수 감소
    group_by_length=True,
    remove_unused_columns=True,
    load_best_model_at_end=True,

    # 추가 설정
    ddp_find_unused_parameters=False,  # DDP 최적화
    torch_compile=False,               # 컴파일 비활성화로 안정성 향상
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.4 MB/s eta 0:00:00


In [ ]:
# 트레이너 설정
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    peft_config=peft_config  # 로라 설정 SFTTrainer 내부에 적용
)

Converting train dataset to ChatML:   0%|          | 0/2000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/200 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# 학습 실행
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
50,1.627700,1.591480
100,1.496900,1.527893


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=125, training_loss=1.5989828491210938, metrics={'train_runtime': 663.8642, 'train_samples_per_second': 3.013, 'train_steps_per_second': 0.188, 'total_flos': 4.3708833595392e+16, 'train_loss': 1.5989828491210938})

### 모델 평가하기

In [ ]:
!pip install rouge_score -qqq
import numpy as np
import torch
from rouge_score import rouge_scorer
from tqdm.auto import tqdm

def evaluate_model_metrics(model, tokenizer, test_dataset, batch_size=8):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)  # ROUGE 점수 계산 초기화

    # 평가 결과 저장용 리스트
    rouge_scores = {
        'rouge1': [],
        'rouge2': [],
        'rougeL': []
    }

    # 모델을 평가 모드로 변경
    model.eval()

    # 배치 단위로 데이터 처리
    num_batches = (len(test_dataset) + batch_size - 1) // batch_size  # 배치 개수 계산

    for i in tqdm(range(num_batches), desc="Evaluating"):  # 배치 단위 데이터 추출
        batch_start = i * batch_size
        batch_end = min((i + 1) * batch_size, len(test_dataset))
        batch_documents = test_dataset[batch_start:batch_end]

        # 배치 프롬프트 생성
        prompts = [
            f"[INST] 아래 뉴스 기사를 요약해주세요:\n\n기사 내용:\n{doc['document']}\n\n요약: [/INST]"
            for doc in batch_documents
        ]
        references = [doc['summary'] for doc in batch_documents]

        # 추론할 때 토크나이저 패딩 방향 설정 (generate() 사용 시)
        tokenizer.padding_side = "left"  # ✅ 추론 시에는 left-padding 사용

        # 입력을 토큰화 후 GPU로 이동
        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=2048).to(model.device)

        # 모델 예측 (배치 처리)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=128, do_sample=True, temperature=0.7, pad_token_id=tokenizer.eos_token_id)

        # 모델이 생성한 요약문 디코딩 (배치 단위 처리)
        predicted_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predicted_summaries = [pred.split("[/INST]")[-1].strip() for pred in predicted_summaries]

        # ROUGE 점수 계산 (배치 단위 처리)
        for reference, predicted in zip(references, predicted_summaries):
            scores = scorer.score(reference, predicted)
            rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
            rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
            rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

    # 평균 ROUGE 점수 계산
    avg_scores = {
        'rouge1': np.mean(rouge_scores['rouge1']),
        'rouge2': np.mean(rouge_scores['rouge2']),
        'rougeL': np.mean(rouge_scores['rougeL'])
    }

    # 결과 출력
    print(f"ROUGE-1: {avg_scores['rouge1']:.4f}")
    print(f"ROUGE-2: {avg_scores['rouge2']:.4f}")
    print(f"ROUGE-L: {avg_scores['rougeL']:.4f}")

    return avg_scores

In [ ]:
# test_dataset을 리스트로 변환
test_data_list = test_dataset.to_list()

# 평가 실행 (배치 크기 8로 설정)
scores = evaluate_model_metrics(model, tokenizer, test_data_list, batch_size=8)

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

ROUGE-1: 0.4150
ROUGE-2: 0.1951
ROUGE-L: 0.3971


### LoRA 모델 저장 및 허깅페이스 허브 업로드

In [ ]:
# LoRA 모델 저장
trainer.model.save_pretrained("./mistral-finetuned")
tokenizer.save_pretrained("./mistral-finetuned")

('./mistral-finetuned/tokenizer_config.json',
 './mistral-finetuned/special_tokens_map.json',
 './mistral-finetuned/tokenizer.model',
 './mistral-finetuned/added_tokens.json',
 './mistral-finetuned/tokenizer.json')

In [ ]:
# Hugging Face 로그인
login()  # 또는 'huggingface-cli login' 실행 후 토큰 입력

# LoRA 가중치 병합
trainer.model = trainer.model.merge_and_unload()  # LoRA 병합

# Hugging Face에 업로드할 모델 ID
repo_id = "edgeun/mistral-7b-instruct-v0.1-korean-news-summarizer"

# 병합된 모델 업로드
trainer.model.push_to_hub(repo_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.push_to_hub(repo_id)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


model.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/edgeun/mistral-7b-instruct-v0.1-korean-news-summarizer/commit/c84753e68d104b75fba4c94d3f89ac74c3e7e488', commit_message='Upload tokenizer', commit_description='', oid='c84753e68d104b75fba4c94d3f89ac74c3e7e488', pr_url=None, repo_url=RepoUrl('https://huggingface.co/edgeun/mistral-7b-instruct-v0.1-korean-news-summarizer', endpoint='https://huggingface.co', repo_type='model', repo_id='edgeun/mistral-7b-instruct-v0.1-korean-news-summarizer'), pr_revision=None, pr_num=None)

### 업로드한 모델 불러와서 사용해보기

In [ ]:
!pip install bitsandbytes -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.7 MB/s eta 0:00:00


In [ ]:
import bitsandbytes
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

my_model_id = "edgeun/mistral-7b-instruct-v0.1-korean-news-summarizer"

# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(my_model_id, device_map="auto")

# 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(my_model_id)

# 패딩 토큰 설정
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"  # 추론 시 패딩 방향 '왼쪽'

# 모델을 추론 모드로 변경
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

In [ ]:
# 추론 함수
def generate_summary(article):
    prompt = f"[INST] 아래 뉴스 기사를 요약해주세요:\n\n기사 내용:\n{article}\n\n요약: [/INST]"

    # 토큰화
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

    # 모델 추론
    output = model.generate(**inputs, max_new_tokens=128, num_beams=3, do_sample=True, temperature=0.7)

    # 결과 디코딩
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    summary = summary.split("[/INST]")[-1].strip()  # 요약문만 출력

    return summary

In [ ]:
# 테스트 실행
article = """

정월대보름(2월12일)을 앞두고 서민들의 시름이 깊어지고 있다.
고물가 장기화에 장바구니 가격 부담이 갈수록 커지는 가운데 정월대보름에 챙겨먹는 오곡밥과 부럼 등 재료 가격마저 크게 올랐기 때문이다. 대형마트는 국산 재료 값이 급등하자 일부 품목을 수입산으로 대체하고 있다.
10일 대형마트 업계에 따르면 오곡밥 주재료인 붉은팥, 찹쌀, 서리태, 수수, 차조 등 국산 잡곡 시세가 일제히 상승했다.
특히 잡곡밥에 들어가는 붉은팥 가격이 전년 대비 50%가량 뛰었고 찹쌀도 23% 이상 급등했다. 부럼 재료인 은행과 땅콩 가격 역시 17%가량 올랐다. 국산 건나물도 상황은 마찬가지다.
호박과 고구마순의 가격이 각각 20%, 10% 이상 뛰었고 기획상품으로 내놓는 건나물 4종 세트 역시 전년 대비 평균 5~10% 올랐다.
유통업계에서는 정월대보름 주요 품목 가격이 오른 이유로 재배 면적 축소에 따른 생산량 감소, 폭염 등 이상기후로 인한 작황 부진, 고물가 장기화에 집밥 수요 급증 등의 영향 등을 꼽고 있다.
이에 따라 대형마트들은 고객들의 장바구니 물가 부담을 덜어주기 위해 일부 품목을 수입산으로 대체하고 있다.
붉은팥과 호두, 땅콩 등이 대표적이다.

"""

summary = generate_summary(article)
print("뉴스 요약:", summary)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


뉴스 요약: 10일 대형마트 업계에 따르면 오곡밥 주재료인 붉은팥, 찹쌀, 서리태, 수수, 차조 등 국산 재료 값이 급등하자 일부 품목을 수입산으로 대체하고 있다. 특히 잡곡밥에 들어가는 붉은팥 가격이
